<a href="https://colab.research.google.com/github/thaisrezendeb/ufes_machineLearning/blob/main/L2_Exercicio5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**5) Para a base Car Evaluation (disponível em http://archive.ics.uci.edu/ml/) obtenha as regras de classificação usando o algoritmo de cobertura top-down (use a taxa de erro como função de avaliação).**

In [ ]:
import pandas as pd

uri =  "https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data"
car = pd.read_csv(uri, header=None)

#Atributes:
# 0 = buying: vhigh, high, med, low
# 1 = maint: vhigh, high, med, low
# 2 = doors: 2, 3, 4, 5more
# 3 = persons: 2, 4, more
# 4 = lug_boot: small, med, big
# 5 = safety: low, med, high
# 6 = class = unacc, acc, good, vgood 

columns_car = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']

def error_min(data, classe):
  min_i = data.shape[1]
  min_a = 'erro'
  min_erro = 1

  #Testa para cada atributo exceto 'class'
  for i in range(0, data.shape[1]-1):
    for a in data[i].unique():
      data_a = data[data[i] == a]
      erro = 1 - data_a[data_a[6] == classe].shape[0]/data_a.shape[0]
      
      if erro < min_erro:
        min_erro = erro
        min_a = a
        min_i = i
  
  return min_i, min_a, min_erro

In [ ]:
regras = pd.DataFrame(columns=["Classe", "Atributo", "Regra (=)", "Erro"])

#Para cada classe
for c in car[6].unique():
  #Erro sem nenhuma regra
  last_err = 1 - car[car[6] == c].shape[0]/car.shape[0]
  
  car_copy = car
  continua = True

  while continua:
    col, val, err = error_min(car_copy, c)
    
    if err < last_err: #Regra reduziu o erro
      last_err = err
      
      #Adiciona na lista de regras
      regras.loc[len(regras)] = [c, columns_car[col], val, err]

      #Apaga amostras cobertas pela regra atual
      car_copy = car_copy.drop(car_copy[car_copy[col] == val].index, axis=0)

      if car_copy.shape[0] == 0: #Não há mais amostras para cobrir
        continua = False
    else:
      #Nenhuma regra melhora a cobertura, precisa adicionar mais uma opção
      continua = False

regras

,Classe,Atributo,Regra (=),Erro
0,unacc,persons,2,0.000000
1,acc,safety,high,0.645833
2,vgood,safety,high,0.887153
3,good,buying,low,0.893519


In [ ]:
  #Sem restrição
  car_c = car[car[6] == c]
  erro = 1 - car_c.shape[0]/car.shape[0]
  min_erro = erro
  continua = True
  count = 0

  while continua:
    min_a = c
    min_i = 6

    
    print("Erro mínimo classe '{}': atributo '{}' | valor '{}' | erro = {:.2f}%" \
                .format(c, columns_car[min_i], min_a, min_erro*100))
  
    count = count + 1
    if car_c.shape[0] == 0 or min_erro < 0.1 or count > 10:
      continua = False

print("")    
regras